In [1]:

import torch
import os
import matplotlib.pyplot as plt

from torchvision.io import read_image, write_png
from torch.utils.cpp_extension import load_inline
from pathlib import Path

In [2]:
ROOT_DIR = os.path.abspath("..")
IMAGES_DIR = Path(ROOT_DIR) / "images"
BUILD_DIR = Path(ROOT_DIR) / "build"
KERNELS_DIR = Path(ROOT_DIR) / "kernels"
IMAGES_DIR


PosixPath('/home/aleksei/vs_code/cuda_torch_learning/pmpp/images')

### Image Blur

In [2]:
cuda_source = (KERNELS_DIR / "imageBlur.cu").read_text()
cpp_source = "torch::Tensor blur_filter(torch::Tensor image, int radius);"
image_blur_module = load_inline(
    name="image_blur",
    cpp_sources=[cpp_source],
    cuda_sources=[cuda_source],
    functions=["blur_filter"],
    build_directory=BUILD_DIR,
    verbose=True,
)

/home/aleksei/vs_code/cuda_torch_learning/pmpp/jupyter
